In [1]:
import re
import base64
from openai import OpenAI
from anthropic import Anthropic
from pdf2image import convert_from_path

from pathlib import Path
from tqdm import tqdm
import pandas as pd
import json

import os
os.getcwd()

'/Users/roshansk/Documents/GitHub/Global_Exams'

In [2]:
def encode_image(image_path):
    """
    Function to encode the image
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def get_message_list(base64_string, prompt):

    message_list = [
    {
        "role": 'user',
        "content": [
            {"type": "image", "source": {"type": "base64", "media_type": "image/jpeg", "data": base64_string}},
            {"type": "text", "text": prompt}
        ]
    }
    ]

    return message_list

In [3]:
from anthropic import Anthropic

client = Anthropic()

# MODEL_NAME = "claude-3-opus-20240229"

MODEL_NAME = "claude-3-5-sonnet-20240620"




In [49]:


pdf_file = '/Users/roshansk/Documents/GitHub/Global_Exams/data/2020/NEET_2020_Marathi_G3.pdf'
answer_file = '/Users/roshansk/Documents/GitHub/Global_Exams/data/2020/NEET_2020_G3_Answer_Key.json'
image_folder = '/Users/roshansk/Documents/GitHub/Global_Exams/images/NEET_2020_Marathi_G3'


lang_code = 'mr'
country = 'India'
file_name = 'Paper_20201106084438.pdf'
source = 'https://www.nta.ac.in/Download/ExamPaper/Paper_20201106084438.pdf'
license = 'open'
level = 'University'

output_file_name = 'NEET_2020_Marathi_processed'

pages_to_include=[2,21]

## Prompt (User input required. Change the prompt value to account for language)

In [11]:
# prompt = """Extract the multiple choice questions along with the options present in the image. The text is mostly in the language of {lang}.
# Return the extracted the questions as a list of json values, with each json value corresponding to a single question. 
# The json value should have the following keys : 
# - number : The number of the question
# - question : The actual text of the question
# - options : A list containing all 4 options for the question
# - image : output True if there is an image associated with the either the question or answer and the student is supposed to use an image to answer the question. Output False, otherwise. Ensure that True and False have their first letters capitalized.

# Output the list alone and no other supporting text in your response. Ensure that the output can be converted into a python directly using eval(response.text)
# The questions are present in 2 columns. Make sure to extract questions from both columns and not just one

# """

# prompt_hindi = """You are given a pdf containing Hindi and English questions. Extract the Hindi multiple choice questions along with the options present in the image.
# Return the extracted the questions as a list of json values, with each json value corresponding to a single question. 
# The json value should have the following keys : 
# - number : The number of the question
# - question : The actual text of the question
# - options : A list containing all 4 options for the question
# - image : output True if there is an image associated with the either the question or answer and the student is supposed to use an image to answer the question. Output False, otherwise. Ensure that True and False have their first letters capitalized.

# Output the list alone and no other supporting text in your response. Ensure that the output can be converted into a python directly using eval(response.text)
# The questions are present in two columns with Hindi questions on the left column and English questions on the right column. Extract only the Hindi questions from the left column.
# """


# prompt = """You are given a pdf containing Gujarati and English questions. Extract the Gujarati multiple choice questions along with the options present in the image.
# Return the extracted the questions as a list of json values, with each json value corresponding to a single question. 
# The json value should have the following keys : 
# - number : The number of the question
# - question : The actual text of the question
# - options : A list containing all 4 options for the question
# - image : output True if there is an image associated with the either the question or answer and the student is supposed to use an image to answer the question. Output False, otherwise. Ensure that True and False have their first letters capitalized.

# Output the list alone and no other supporting text in your response. Ensure that the output can be converted into a python dict directly using eval(response.text)
# The questions are present in two columns with Gujarati questions on the right side and English questions on the left. Extract only the Gujarati questions from the right column.
# """


prompt = """Extract the multiple choice questions along with the options present in the image. The text is mostly in the language of Marathi.
Return the extracted the questions as a list of json values, with each json value corresponding to a single question. 
The json value should have the following keys : 
- number : The number of the question
- question : The actual text of the question
- options : A list containing all 4 options for the question
- image : output True if there is an image or table associated with the either the question or answer and the student is supposed to use an image/table to answer the question. Output False, otherwise. Ensure that True and False have their first letters capitalized.

Output the list alone and no other supporting text in your response. Ensure that the output can be converted into a python dict directly using eval(response.text)
The questions are present in 2 columns. Make sure to extract questions from both columns and not just one

"""

## Creating Image Files

In [50]:
if not os.path.exists(image_folder):
    os.makedirs(image_folder)

pages = convert_from_path(pdf_file)

start_page, end_page = pages_to_include 
count = 0
for i in range(start_page-1,  end_page):
    filename = "page_"+str(i)+'.jpg'
    pages[i].save(Path(image_folder) / filename)
    count += 1
    

In [51]:
print(f"{count} images created")

20 images created


## Extracting questions from Claude

In [52]:
imageList = os.listdir(image_folder)

json_output = {}
text_output = {}

for i in tqdm(range(len(imageList))):
    image_path = Path(image_folder) / imageList[i]
    base64_string = encode_image(image_path)

    message_list = get_message_list(base64_string, prompt) 

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        messages=message_list,
        temperature = 0.0,
        top_p = 1
        
    )

    try:
        out = eval(response.content[0].text)

        json_output[i] = out 
        print(f"{imageList[i]} added to json")
    except:
        text_output[i] = response.content[0].text
        print(f"{imageList[i]} added to text")
    
json_backup = json_output.copy()


  5%|██▏                                         | 1/20 [00:31<09:49, 31.00s/it]

page_2.jpg added to json


 10%|████▍                                       | 2/20 [00:57<08:35, 28.63s/it]

page_3.jpg added to json


 15%|██████▌                                     | 3/20 [01:24<07:50, 27.68s/it]

...

page_14.jpg added to json


 95%|████████████████████████████████████████▊  | 19/20 [11:53<00:27, 27.59s/it]

page_8.jpg added to json


100%|███████████████████████████████████████████| 20/20 [12:12<00:00, 36.60s/it]

page_9.jpg added to json


In [53]:
output = []
for key in json_output.keys():
    output += json_output[key]


df = pd.DataFrame(output)
df.columns = ['original_question_num','question','options','image']

print(f"Total no of samples : {len(df)}")
print(f"Total no of samples without images: {len(df[df.image==False])}")
df.head()

Total no of samples : 171
Total no of samples without images: 149


,original_question_num,question,options,image
0,10,वनस्पतीत खालीलपैकी कोणता शरीरभाग दोन पिढ्या एक...,"[परागकोशातील परागकण, अंकुरित झालेल्या परागकण व...",False
1,11,प्लाझ्मोडिअमची मानवी शरीरात प्रवेशणारी संक्रमण...,"[स्पोरोझोइट्स, मादी युग्मकेशी (मादी गॅमिटोसाइट...",False
2,12,अयोग्य विधान ओळखा :,[रक्तकाष्ठ पाणी व खनिजांचे वहन मूळापासून पानां...,False
3,13,पेंसीनचे आणि अॅल्गिननचे पिसपूर ही उदाहरणे ____...,"[समकेंद्री उष्णती, ओलीगोसॅकरायडला, नैसर्गिक नि...",False
4,14,जीन 'I' जो ABO रक्तगट नियंत्रण करतो त्याच्याशी...,[एका व्यक्तीमध्ये तीन पैकी दोन युग्मविकल्प असत...,False


#### Removing samples with images/tables

In [54]:
df = df[df.image == False]

#### Attaching Answer Key

In [55]:
with open(answer_file) as f:
    answer_key = json.load(f)

In [56]:
df.original_question_num = df.original_question_num.apply(lambda x: x.replace(".","")) #Cleaning

df['answer'] = df.original_question_num.apply(lambda x : str(answer_key[x]))

#### Assigning categories

In [57]:
category_map = {'biology': 'जीवशास्त्र' ,'chemistry':'रसायनशास्त्र','physics':'भौतिकशास्त्र'}

question_category_map = {}
for i in range(1,91):
    question_category_map[i] = 'biology'

for i in range(91,136):
    question_category_map[i] = 'physics'

for i in range(136,181):
    question_category_map[i] = 'chemistry'

In [58]:
df['category_en'] = df.original_question_num.apply(lambda x : question_category_map[int(x)])
df['category_original_lang'] = df.category_en.apply(lambda x : category_map[x])

#### Assigning other metadata

In [59]:
df['language'] = lang_code
df['country'] = country
df['file_name'] = file_name
df['source'] = source
df['license'] = license
df['level'] = level


df_ = df.copy()
df.drop('image', axis = 1, inplace = True)

cols = ['language',
       'country', 'file_name', 'source', 'license', 'level', 'category_en',
       'category_original_lang', 'original_question_num', 'question', 'options', 'answer']

df = df[cols]

In [60]:
df.head()

,language,country,file_name,source,license,level,category_en,category_original_lang,original_question_num,question,options,answer
0,mr,India,Paper_20201106084438.pdf,https://www.nta.ac.in/Download/ExamPaper/Paper...,open,University,biology,जीवशास्त्र,10,वनस्पतीत खालीलपैकी कोणता शरीरभाग दोन पिढ्या एक...,"[परागकोशातील परागकण, अंकुरित झालेल्या परागकण व...",3
1,mr,India,Paper_20201106084438.pdf,https://www.nta.ac.in/Download/ExamPaper/Paper...,open,University,biology,जीवशास्त्र,11,प्लाझ्मोडिअमची मानवी शरीरात प्रवेशणारी संक्रमण...,"[स्पोरोझोइट्स, मादी युग्मकेशी (मादी गॅमिटोसाइट...",1
2,mr,India,Paper_20201106084438.pdf,https://www.nta.ac.in/Download/ExamPaper/Paper...,open,University,biology,जीवशास्त्र,12,अयोग्य विधान ओळखा :,[रक्तकाष्ठ पाणी व खनिजांचे वहन मूळापासून पानां...,2
3,mr,India,Paper_20201106084438.pdf,https://www.nta.ac.in/Download/ExamPaper/Paper...,open,University,biology,जीवशास्त्र,13,पेंसीनचे आणि अॅल्गिननचे पिसपूर ही उदाहरणे ____...,"[समकेंद्री उष्णती, ओलीगोसॅकरायडला, नैसर्गिक नि...",1
4,mr,India,Paper_20201106084438.pdf,https://www.nta.ac.in/Download/ExamPaper/Paper...,open,University,biology,जीवशास्त्र,14,जीन 'I' जो ABO रक्तगट नियंत्रण करतो त्याच्याशी...,[एका व्यक्तीमध्ये तीन पैकी दोन युग्मविकल्प असत...,2


#### Save processed file

In [61]:
df.to_json(output_file_name+'.json', orient='records')
df_.to_csv(output_file_name+'.tsv',sep='\t')

In [48]:
len(df)

161